In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import generative as gen
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from tqdm import tqdm
from scipy.spatial.distance import pdist, squareform
import utils as ut
import seaborn as sns
from vizman import viz
from scipy import io
import pandas as pd
from neuromaps.datasets import fetch_fslr
from surfplot import Plot
from netneurotools.networks import struct_consensus


/Users/kf02/Library/Mobile Documents/com~apple~CloudDocs/Work/Optimal_networks/.venv/lib/python3.11/site-packages/tqdm_joblib/__init__.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
viz.set_visual_style()
viz_sizes = viz.load_data_from_json("sizes.json")
viz_colors = viz.load_data_from_json("colors.json")
viz_cmaps = viz.give_colormaps()
sns_kwargs = {"cmap": viz_cmaps["bw_lr"],
              "xticklabels":False,
              "yticklabels":False,
              "rasterized":True}

surfcolors = sns.blend_palette([
                                viz_colors["neutrals"]["ALL_WHITE"],
                                viz_colors["warms"]["LECKER_RED"]],
                                     as_cmap=True)

In [4]:
res_parcellation = 1
consensus_mat = io.loadmat(
    "datasets/human/Consensus_Connectomes.mat",
    simplify_cells=True,
    squeeze_me=True,
    chars_as_strings=True,
)
connectivity = consensus_mat["LauConsensus"]["Matrices"][res_parcellation][0].astype(bool)
coordinates = consensus_mat["LauConsensus"]["Matrices"][res_parcellation][3]
labels = consensus_mat["LauConsensus"]["Matrices"][res_parcellation][-1]
euclidean_dis = squareform(pdist(coordinates, metric='euclidean'))
all_connectivities = io.loadmat(
    "datasets/human/Individual_Connectomes.mat",
    simplify_cells=True,
    squeeze_me=True,
    chars_as_strings=True,
)['connMatrices']['SC'][res_parcellation].astype(bool)

In [5]:
n_iterations = 3000
n_nodes = labels.shape[0]
beta = np.full(n_iterations, 1)
noise = np.zeros(n_iterations)
penalty = np.zeros(n_iterations)
batch_size = np.full(n_iterations, 18)

In [36]:
non_normalized_diffusion_matrices = gen.simulate_network_evolution(
        coordinates=coordinates,
        n_iterations=n_iterations,
        alpha=np.full(n_iterations, 8),
        beta=np.full(n_iterations, 1),
        noise=noise,
        distance_fn=gen.resistance_distance,
        connectivity_penalty=penalty,
        n_jobs=-1,
        random_seed=11,
        batch_size=batch_size,
    )


Simulating network evolution: 100%|██████████| 2999/2999 [01:00<00:00, 49.78it/s]


In [37]:
ut.check_density(connectivity), ut.check_density(non_normalized_diffusion_matrices[:,:,-1])

(0.20276354603322466, 0.19251668995497592)

In [38]:
ut.check_density(connectivity)-ut.check_density(non_normalized_diffusion_matrices[:,:,-1])

0.01024685607824874